## [[Discourse] AttributeError: 'DataFrame' object has no attribute 'repartition' #207](https://github.com/coiled/dask-community/issues/207)

In [9]:
import dask
import torch

import pandas as pd
import dask.dataframe as dd

In [9]:
df = pd.DataFrame({'userID': ['1', '2', '3'], 'questionID': ['a', 'b', 'c'], 'result': ['99', '98', '97']})

ddf = dd.from_pandas(df, npartitions=2)

In [9]:
MAX_LEN = 300
users = ddf.userID.unique().compute()
paths = []
labels = []

In [9]:
@dask.delayed(nout=2)
def get_paths(df, user):
    temp = df.loc[df.userID == user]
    npart = round(len(temp)/MAX_LEN)
    if npart == 0:
        return (temp.questionID.values, temp.result.values)
    else:
        parted_df = temp.repartition(npartitions=npart)
        return (parted_df.partitions[0].questionID.values,
                parted_df.partitions[0].result.values)

In [9]:
@dask.delayed
def convert_to_tensor(x):
    return x

In [9]:
for user in users:
    path, label = get_paths(ddf, user)
    paths.append(convert_to_tensor(path))
    labels.append(convert_to_tensor(label))

In [9]:
dask.compute(*paths)

In [9]:
dask.compute(*labels)